In [1]:
#importing all libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from google.colab import files


In [2]:
#reading input data

data=pd.read_excel("/content/Input.xlsx")

In [3]:
data.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [4]:
data.count()

URL_ID    114
URL       114
dtype: int64

In [5]:
#importing csv file to save files in csv format

import csv

extracted_text = []
url_id = 37
 # creating loop to extract text from giver urls

for i in range(0, len(data)):
    j = data.iloc[i].values
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
    }
    page = requests.get(j[1], headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    content = soup.findAll(attrs={'class': 'td-post-content'})
    title = soup.findAll(attrs={'class': 'entry-title'})
    tittle = title[16].text.replace('\n', '  ').replace('/', '')
    text = tittle + '.' + str(content)
    extracted_text.append(text)

csv_file = "extracted_text.csv"
Data = [{'url_id': i, 'Text': text} for i, text in enumerate(extracted_text, start=url_id)]
header = ['url_id', 'Text']

with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    writer.writerows(Data)

print("Text extraction completed. The extracted text is saved in 'extracted_text.csv' file.")


Text extraction completed. The extracted text is saved in 'extracted_text.csv' file.


In [6]:
#importing each extracted files

text=pd.read_csv("/content/extracted_text.csv")
text.head()

,url_id,Text
0,37,Rise of telemedicine and its Impact on Livelih...
1,38,What if the Creation is Taking Over the Creato...
2,39,What Jobs Will Robots Take From Humans in The ...
3,40,Will Machine Replace The Human in the Future o...
4,41,Will AI Replace Us or Work With Us?.[<div clas...


In [7]:
#information of data frame

text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url_id  114 non-null    int64 
 1   Text    114 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [8]:
#converting type

text=text.astype(str)

In [9]:
#converting text to sentence

import re
a=text.replace('\n',"  ").replace('/',"")
b=pd.DataFrame(a)#creating data frame


In [10]:
b.head()

,url_id,Text
0,37,Rise of telemedicine and its Impact on Livelih...
1,38,What if the Creation is Taking Over the Creato...
2,39,What Jobs Will Robots Take From Humans in The ...
3,40,Will Machine Replace The Human in the Future o...
4,41,Will AI Replace Us or Work With Us?.[<div clas...


In [11]:
#removing . char from each rows

def abcd(x):
    nopunc =[char for char in x if char != '.']
    return ''.join(nopunc)
b['Text']=b['Text'].apply(abcd)


In [12]:
#replacing emty space with null values

c=b.replace('',np.nan,regex=True)
c=c.mask(c==" ")
c=c.dropna()
c.reset_index(drop=True,inplace=True)

In [74]:
c.head()

,url_id,Text
0,37,Rise of telemedicine and its Impact on Livelih...
1,38,What if the Creation is Taking Over the Creato...
2,39,What Jobs Will Robots Take From Humans in The ...
3,40,Will Machine Replace The Human in the Future o...
4,41,Will AI Replace Us or Work With Us[<div class=...


In [14]:
#importing nltk library and stopwords

import nltk
import string

In [15]:
#punctuations

punc=[punc for punc in string.punctuation]

In [16]:
punc

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [17]:
#importing stop words files that are provided

StopWords_Auditor=pd.read_csv("/content/StopWords_Auditor.txt",header=None)
StopWords_Currencies=pd.read_csv("/content/StopWords_Currencies.txt",encoding='latin-1',error_bad_lines=False)
StopWords_DatesandNumbers=pd.read_csv("/content/StopWords_DatesandNumbers.txt",header=None)
StopWords_Generic=pd.read_csv("/content/StopWords_Generic.txt",header=None)
StopWords_GenericLong=pd.read_csv("/content/StopWords_GenericLong.txt",header=None)
StopWords_Geographic=pd.read_csv("/content/StopWords_Geographic.txt",header=None)
StopWords_Names=pd.read_csv("/content/StopWords_Names.txt",header=None)


<ipython-input-17-b0929ba96e22>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  StopWords_Currencies=pd.read_csv("/content/StopWords_Currencies.txt",encoding='latin-1',error_bad_lines=False)
Skipping line 83: expected 1 fields, saw 2



In [18]:
#creating func for removing stop words

def text_process(text):
    nopunc =[char for char in text if char not in punc or char not in [':',',','(',')','’','?']]
    nopunc=''.join(nopunc)
    txt=' '.join([word for word in nopunc.split() if word.lower() not in StopWords_Auditor])
    txt1=' '.join([word for word in txt.split() if word.lower() not in StopWords_Currencies])
    txt2=' '.join([word for word in txt1.split() if word.lower() not in StopWords_DatesandNumbers])
    txt3=' '.join([word for word in txt2.split() if word.lower() not in StopWords_Generic])
    txt4=' '.join([word for word in txt3.split() if word.lower() not in StopWords_GenericLong])
    txt5=' '.join([word for word in txt4.split() if word.lower() not in StopWords_Geographic])
    return ' '.join([word for word in txt5.split() if word.lower() not in StopWords_Names])

In [19]:
#applying func for each row

c['Text']=c['Text'].apply(text_process)

In [75]:
c.head()

,url_id,Text
0,37,Rise of telemedicine and its Impact on Livelih...
1,38,What if the Creation is Taking Over the Creato...
2,39,What Jobs Will Robots Take From Humans in The ...
3,40,Will Machine Replace The Human in the Future o...
4,41,Will AI Replace Us or Work With Us[<div class=...


In [23]:
#importing master Dictionary

positive=pd.read_csv("/content/positive-words.txt",header=None)
negative=pd.read_csv("/content/negative-words.txt",header=None,encoding="ISO-8859-1",error_bad_lines=False )

<ipython-input-23-8fd54805983e>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  negative=pd.read_csv("/content/negative-words.txt",header=None,encoding="ISO-8859-1",error_bad_lines=False )


In [24]:
#positive and negative dictionary with stopwords

positive.columns=['Text']
negative.columns=['Text']
positive['Text']=positive['Text'].astype(str)
negative['Text']=negative['Text'].astype(str)

In [25]:
#positive and negative dictionary without stopwords
positive['Text']=positive['Text'].apply(text_process)
negative['Text']=negative['Text'].apply(text_process)

In [26]:
#positive list

length=positive.shape[0]
post=[]
for i in range(0,length):
   nopunc =[char for char in positive.iloc[i] if char not in string.punctuation or char != '+']
   nopunc=''.join(nopunc)

   post.append(nopunc)

In [27]:
#negative list

length=negative.shape[0]
neg=[]
for i in range(0,length):
  nopunc =[char for char in negative.iloc[i] if char not in string.punctuation or char != '+']
  nopunc=''.join(nopunc)
  neg.append(nopunc)

In [28]:
#importing tokenize library

from nltk.tokenize import word_tokenize

In [29]:
txt_list=[]
length=c.shape[0]

for i in range(length):
    txt = ' '.join([str(word) for word in c.iloc[i]])
    txt_list.append(txt)

In [64]:
#installing nltk

!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
#downloading punkt from nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [62]:
#installing textstatistics library

!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
#computing variables for each text


import pandas as pd
import nltk
from textblob import TextBlob
from textstat import textstat

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Function to compute variables from text
def compute_variables(c):
    # Sentiment analysis
    sentiment = TextBlob(c)
    positive_score = sentiment.sentiment.polarity
    negative_score = sentiment.sentiment.subjectivity
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)

    # Tokenization and sentence count
    tokens = nltk.word_tokenize(c)
    sentences = nltk.sent_tokenize(c)
    sentence_count = len(sentences)

    # Average sentence length
    avg_sentence_length = len(tokens) / sentence_count if sentence_count > 0 else 0

    # Percentage of complex words
    complex_word_count = textstat.difficult_words(c)
    percentage_complex_words = (complex_word_count / len(tokens)) * 100 if len(tokens) > 0 else 0

    # Fog index
    fog_index = textstat.gunning_fog(c)

    # Average number of words per sentence
    avg_no_of_words_per_sentence = len(tokens) / sentence_count if sentence_count > 0 else 0

    # Word count
    word_count = len(tokens)

    # Syllable count
    syllable_count = textstat.syllable_count(c)

    # Personal pronouns count
    tagged_tokens = nltk.pos_tag(tokens)
    personal_pronouns = len([word for word, tag in tagged_tokens if tag == 'PRP'])

    # Average word length
    avg_word_length = sum(len(word) for word in tokens) / len(tokens) if len(tokens) > 0 else 0

    # Return computed variables as a dictionary
    variables = {
        'positive_score': positive_score,
        'negative_score': negative_score,
        'Polarity_Score': polarity_score,
        'subjectivity_score': negative_score,
        'avg_sentence_length': avg_sentence_length,
        'Percentage_of_Complex_words': percentage_complex_words,
        'Fog_Index': fog_index,
        'avg_no_of_words_per_sentence': avg_no_of_words_per_sentence,
        'complex_Word_Count': complex_word_count,
        'word_count': word_count,
        'syllable_count': syllable_count,
        'personal_pronouns': personal_pronouns,
        'avg_word_length': avg_word_length
    }

    return variables

# Load the DataFrame with the text column

df = pd.read_csv('/content/extracted_text.csv')

# Apply text analysis and compute variables on the text column

computed_variables = df['Text'].apply(compute_variables)

# Create a new DataFrame with the computed variables

output_df = pd.DataFrame(computed_variables.tolist())

# Concatenate the original DataFrame with the computed variables DataFrame

df_with_variables = pd.concat([df, output_df], axis=1)

# Print the DataFrame with the computed variables

print(df_with_variables)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


     url_id                                               Text  \
0        37  Rise of telemedicine and its Impact on Livelih...   
1        38  What if the Creation is Taking Over the Creato...   
2        39  What Jobs Will Robots Take From Humans in The ...   
3        40  Will Machine Replace The Human in the Future o...   
4        41  Will AI Replace Us or Work With Us?.[<div clas...   
..      ...                                                ...   
109     146  Blockchain for Payments.[<div class="td-post-c...   
110     147  The future of Investing.[<div class="td-post-c...   
111     148  Big Data Analytics in Healthcare.[<div class="...   
112     149  Business Analytics In The Healthcare Industry....   
113     150  Challenges and Opportunities of Big Data in He...   

     positive_score  negative_score  Polarity_Score  subjectivity_score  \
0          0.141153        0.477331       -0.543550            0.477331   
1          0.085017        0.425429       -0.666891      

In [72]:
df_with_variables.head()

,url_id,Text,positive_score,negative_score,Polarity_Score,subjectivity_score,avg_sentence_length,Percentage_of_Complex_words,Fog_Index,avg_no_of_words_per_sentence,complex_Word_Count,word_count,syllable_count,personal_pronouns,avg_word_length
0,37,Rise of telemedicine and its Impact on Livelih...,0.141153,0.477331,-0.543550,0.477331,57.849057,17.775603,11.62,57.849057,545,3066,4309,21,5.286040
1,38,What if the Creation is Taking Over the Creato...,0.085017,0.425429,-0.666891,0.425429,25.579710,13.767705,10.77,25.579710,243,1765,2184,51,4.145609
2,39,What Jobs Will Robots Take From Humans in The ...,0.111470,0.488338,-0.628314,0.488338,35.106061,17.781614,12.72,35.106061,412,2317,3215,28,4.816142
3,40,Will Machine Replace The Human in the Future o...,0.152492,0.477108,-0.515590,0.477108,27.860759,12.539755,10.22,27.860759,276,2201,2599,49,4.012267
4,41,Will AI Replace Us or Work With Us?.[<div clas...,0.044859,0.531324,-0.844289,0.531324,44.057971,12.828947,10.80,44.057971,390,3040,3677,53,4.828618


In [73]:

# Convert DataFrame to CSV
df_with_variables.to_csv('computed_variables.csv', index=False)

# Download the CSV file
files.download('computed_variables.csv')
print("computed_variables.csv file is downloaded")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

computed_variables.csv file is downloaded
